# Test the Takahasi-Takami method for calculating the M/H2/n system

Idea of this algorithm was first proposed by Takahashi and Takami

>Takahashi Y., Takami Y. A Numerical Method for the Steady-State
Probabilities of a GI/G/c Queuing System in a General Class // J. of the
Operat. Res. Soc. of Japan. 1976. — v. 19, no. 2. — P. 147–157.

This method allows to calculate the system for an arbitrary coefficient of variation of service time. When the coefficient of variation of service time < 1, parameters of the approximating H2-distribution are complex, which does not prevent obtaining meaningful results.
For verification we use simulation.


#### Import necessary modules

In [ ]:
import most_queue.random.distributions as rd
from most_queue.io.tables import probs_print, print_waiting_moments
from most_queue.sim.base import QsSim
from most_queue.theory.fifo.mgn_takahasi import MGnCalc

#### Set paramrameters for the simulation and calculation 

In [5]:
NUM_OF_CHANNELS = 3  # number of channels
ARRIVAL_RATE = 1.0  # arrival rate
b = [2.4, 14.0, 130.0]  # service time moments, where b[0] is mean, b[1] is second raw moment, b[2] is third raw moment
NUM_OF_JOBS = 100000  # number of jobs required for the simulation

### Run calcalculation by Takahasi-Takami method

In [6]:
#  Set parameters for the simulation and calculation
tt = MGnCalc(n=NUM_OF_CHANNELS)
tt.set_sources(l=ARRIVAL_RATE)
tt.set_servers(b)
calc_results = tt.run()

num_of_iter = tt.num_of_iter_

### Running the simulation 

In [7]:
qs = QsSim(NUM_OF_CHANNELS)

# Set the arrival process. M - exponential with intensity ARRIVAL_RATE
qs.set_sources(ARRIVAL_RATE, 'M')
# Set servers with Gamma distribution
gamma_params = rd.GammaDistribution.get_params([b[0], b[1]])
qs.set_servers(gamma_params, 'Gamma')

# Run the simulation
sim_results = qs.run(NUM_OF_JOBS)

Start simulation


Job served:    | 0/100 [00:00<?, ?it/s]1000/100000:   1%|          | 1/100 [00:00<00:00, 101.46it/s]2000/100000:   2%|▏         | 2/100 [00:00<00:01, 97.28it/s] 3000/100000:   3%|▎         | 3/100 [00:00<00:01, 88.47it/s]4000/100000:   4%|▍         | 4/100 [00:00<00:01, 90.49it/s]5000/100000:   5%|▌         | 5/100 [00:00<00:01, 91.49it/s]6000/100000:   6%|▌         | 6/100 [00:00<00:01, 82.85it/s]7000/100000:   7%|▋         | 7/100 [00:00<00:01, 84.70it/s]8000/100000:   8%|▊         | 8/100 [00:00<00:01, 86.08it/s]8000/100000:   9%|▉         | 9/100 [00:00<00:01, 86.40it/s]9000/100000:   9%|▉         | 9/100 [00:00<00:01, 86.40it/s]10000/100000:  10%|█         | 10/100 [00:00<00:01, 86.40it/s]11000/100000:  11%|█         | 11/100 [00:00<00:01, 86.40it/s]12000/100000:  12%|█▏        | 12/100 [00:00<00:01, 86.40it/s]13000/100000:  13%|█▎        | 13/100 [00:00<00:01, 86.40it/s]14000/100000:  14%|█▍        | 14/100 [00:00<00:00, 86.40it/s]15000/100000:  15%|█▌        | 15/100 [00:00<00:0

Simulation is finished



## Print the results

In [ ]:
print(f"Number of iterations of the Takahasi-Takami algorithm: {num_of_iter:^4d}")

print(f"Simulation duration: {sim_results.duration:.5f} sec")
print(f"Calculation duration: {calc_results.duration:.5f} sec")

probs_print(sim_results.p, calc_results.p, 10)
print_waiting_moments(sim_results.w, calc_results.w)

Number of iterations of the Takahasi-Takami algorithm:  32 
Simulation duration: 1.49344 sec
Calculation duration: 0.07320 sec
------------------------------------
      Probabilities of states       
------------------------------------
 #  |      Num      |      Sim      
------------------------------------
 0  |    0.05713    |   0.055633    
 1  |    0.13539    |    0.13352    
 2  |    0.15782    |    0.1531     
 3  |    0.11763    |    0.11369    
 4  |   0.092789    |   0.092445    
 5  |   0.075268    |   0.074337    
 6  |   0.061864    |   0.061902    
 7  |   0.051157    |   0.051885    
 8  |   0.042421    |   0.042777    
 9  |   0.035222    |   0.036124    
------------------------------------


raw moments of waiting time in the system
---------------------------------------------
       #       |      Num      |      Sim      
---------------------------------------------
       1       |    3.1233     |    3.2746     
       2       |    30.668     |    32.616     
 